Instalar o Faker

In [ ]:
pip install faker

In [ ]:
from faker import Faker
import mysql.connector
import random

# Inicializa o Faker
fake = Faker()

# Conectar ao MySQL
connection = mysql.connector.connect(
    host='localhost',  # Ou o IP do seu Docker
    user='root',
    password='rootpassword',
    database='sicooperative'
)
cursor = connection.cursor()

# Gerar dados na tabela 'associado'
num_associados = random.randint(50, 100)  # Entre 50 e 100 associados

for _ in range(num_associados):
    nome = fake.first_name()
    sobrenome = fake.last_name()
    idade = fake.random_int(min=18, max=80)
    email = fake.email()

    cursor.execute(
        "INSERT INTO associado (nome, sobrenome, idade, email) VALUES (%s, %s, %s, %s)",
        (nome, sobrenome, idade, email)
    )

# Gerar dados na tabela 'conta'
num_contas = 0
for i in range(num_associados):
    num_contas_associado = random.randint(1, 3)  # Cada associado tem entre 1 e 3 contas
    for _ in range(num_contas_associado):
        tipo = fake.random_choice(['corrente', 'poupança'])
        data_criacao = fake.date_this_decade()
        
        id_associado = i + 1  # Associando com o ID do associado

        cursor.execute(
            "INSERT INTO conta (tipo, data_criacao, id_associado) VALUES (%s, %s, %s)",
            (tipo, data_criacao, id_associado)
        )
        num_contas += 1

# Gerar dados na tabela 'cartao'
num_cartoes = 0
for i in range(num_contas):
    num_cartao_associado = random.randint(1, 2)  # Cada conta pode ter entre 1 e 2 cartões
    for _ in range(num_cartao_associado):
        num_cartao = fake.credit_card_number()
        nom_impresso = fake.name()
        id_conta = i + 1  # Associando com o ID da conta
        id_associado = fake.random_int(min=1, max=num_associados)  # Associa a um associado qualquer

        cursor.execute(
            "INSERT INTO cartao (num_cartao, nom_impresso, id_conta, id_associado) VALUES (%s, %s, %s, %s)",
            (num_cartao, nom_impresso, id_conta, id_associado)
        )
        num_cartoes += 1

# Gerar dados na tabela 'movimento'
num_movimentos = 0
for i in range(num_cartoes):
    num_movimentos_cartao = random.randint(5, 20)  # Cada cartão pode ter entre 5 e 20 movimentações
    for _ in range(num_movimentos_cartao):
        vlr_transacao = fake.random_number(digits=5)
        des_transacao = fake.bs()  # Descrição fictícia da transação
        data_movimento = fake.date_this_year()
        id_cartao = i + 1  # Associando com o ID do cartão

        cursor.execute(
            "INSERT INTO movimento (vlr_transacao, des_transacao, data_movimento, id_cartao) VALUES (%s, %s, %s, %s)",
            (vlr_transacao, des_transacao, data_movimento, id_cartao)
        )
        num_movimentos += 1

# Commit e fechar conexão
connection.commit()
cursor.close()
connection.close()

print(f"{num_associados} associados, {num_contas} contas, {num_cartoes} cartões e {num_movimentos} movimentações foram inseridos nas tabelas com sucesso!")


In [ ]:
jdbc_url = "jdbc:mysql://localhost:3306/sicooperative"
properties = {
    "user": "root",
    "password": "rootpassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Ler as tabelas no MySQL
df_associado = spark.read.jdbc(url=jdbc_url, table="associado", properties=properties)
df_conta = spark.read.jdbc(url=jdbc_url, table="conta", properties=properties)
df_cartao = spark.read.jdbc(url=jdbc_url, table="cartao", properties=properties)
df_movimento = spark.read.jdbc(url=jdbc_url, table="movimento", properties=properties)


In [ ]:
df = df_associado \
    .join(df_conta, df_associado.id == df_conta.id_associado) \
    .join(df_cartao, df_conta.id == df_cartao.id_conta) \
    .join(df_movimento, df_cartao.id == df_movimento.id_cartao) \
    .select(
        df_associado.nome.alias("nome_associado"),
        df_associado.sobrenome.alias("sobrenome_associado"),
        df_associado.idade.alias("idade_associado"),
        df_movimento.vlr_transacao.alias("vlr_transacao_movimento"),
        df_movimento.des_transacao.alias("des_transacao_movimento"),
        df_movimento.data_movimento.alias("data_movimento"),
        df_cartao.num_cartao.alias("numero_cartao"),
        df_cartao.nom_impresso.alias("nome_impresso_cartao"),
        df_conta.data_criacao.alias("data_criacao_cartao"),
        df_conta.tipo.alias("tipo_conta"),
        df_conta.data_criacao.alias("data_criacao_conta")
    )

# Salvar como CSV
df.write.option("header", "true").csv("/dbfs/tmp/movimento_flat.csv")
